# INDEX
Goal: 다음 분기에 어떤 게임을 설계해야 할까?
1. Data Preprocessing
2. 지역에 따라서 선호하는 게임 장르가 다를까
3. 연도별 게임의 트렌드가 있을까
4. 출고량이 높은 게임에 대한 Analysis 및 Visualization

In [ ]:
# Data preprocessing
import numpy as np
import pandas as pd

# Analysis & Visualization
from scipy.stats import chisquare  
from scipy.stats import chi2_contingency
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# 1.Data Preprocessing

In [ ]:
vgames = pd.read_csv("/content/drive/MyDrive/aibootcamp/vgames2.csv",encoding='utf-8')
vgames = vgames.iloc[:,1:]
vgames

In [ ]:
vgames.shape

In [ ]:
'''
< object >
Name : 게임의 이름입니다.
Platform : 게임이 지원되는 플랫폼의 이름입니다.
Genre : 게임의 장르입니다.
Publisher : 게임을 제작한 회사입니다.

< int or float >
Year : 게임이 출시된 연도입니다.
NA_Sales : 북미지역에서의 출고량입니다.
EU_Sales : 유럽지역에서의 출고량입니다.
JP_Sales : 일본지역에서의 출고량입니다.
Other_Sales : 기타지역에서의 출고량입니다.
'''

## -featrue type 맞춰주기

In [ ]:
vgames.dtypes

### year convert

In [ ]:
def convert_year(year):
  if year <= 20:
    return year + 2000
  if year <= 99 :
    return year + 1900
  return year

In [ ]:
vgames['Year'] = vgames['Year'].apply(convert_year)
vgames['Year']

In [ ]:
vgames['Year'].value_counts()

In [ ]:
vgames.loc[vgames['Year']==2020,'Year'] = 2009

In [ ]:
vgames.loc[vgames['Year']==2017,'Year'] = 2016

In [ ]:
vgames['Year'].value_counts()

In [ ]:
def remove(x):

  if str(x).find('K') != -1:
    x = str(x)[:-1]
    x = float(x)
    return x/1000
  if str(x).find('M') != -1:
    x = str(x)[:-1]
    return float(x)
  return float(x)

In [ ]:
vgames['NA_Sales'] = vgames['NA_Sales'].apply(remove)
vgames['EU_Sales'] = vgames['EU_Sales'].apply(remove)
vgames['JP_Sales'] = vgames['JP_Sales'].apply(remove)
vgames['Other_Sales'] = vgames['Other_Sales'].apply(remove)

In [ ]:
vgames.dtypes

In [ ]:
vgames['Platform'].value_counts()

In [ ]:
vgames['Genre'].value_counts()

In [ ]:
vgames['Publisher'].value_counts()

In [ ]:
vgames['Name'].unique

In [ ]:
vgames = vgames.sort_values(by='Year')
vgames = vgames.reset_index()
vgames = vgames.iloc[:,1:]
vgames

## -결측값

In [ ]:
vgames.isna().sum()

### 중복값으로 결측값 상쇄

In [ ]:
first_vgames= vgames[vgames.duplicated(['Name'],keep='first') == 0]
first_vgames.isna().sum()

In [ ]:
first_vgames.shape

In [ ]:
dupli_vgames = vgames[vgames.duplicated(['Name'],keep='first') == 1]
dupli_vgames.isna().sum()

In [ ]:
dupli_vgames

In [ ]:
range(len(vgames))

In [ ]:
range(len(dupli_vgames))

In [ ]:
dupli_nayear_index = dupli_vgames[dupli_vgames['Year'].isna()==1].index
dupli_nagenre_index = dupli_vgames[dupli_vgames['Genre'].isna()==1].index
dupli_napublisher_index = dupli_vgames[dupli_vgames['Publisher'].isna()==1].index
dupli_napublisher_index

In [ ]:
for i in dupli_nayear_index:
  name = vgames.loc[i]['Name']
  temp_df = vgames[vgames['Name'] == name]
  vgames.at[i,'Year'] = temp_df.iloc[0]['Year']

In [ ]:
for i in dupli_nagenre_index:
  name = vgames.loc[i]['Name']
  temp_df = vgames[vgames['Name'] == name]
  temp_df = temp_df.sort_values(by=['Genre'],na_position='last')
  vgames.at[i,'Genre'] = temp_df.iloc[0]['Genre']

In [ ]:
for i in dupli_napublisher_index:
  name = vgames.loc[i]['Name']
  temp_df = vgames[vgames['Name'] == name]
  temp_df = temp_df.sort_values(by=['Publisher'],na_position='last')
  vgames.at[i,'Publisher'] = temp_df.iloc[0]['Publisher']

In [ ]:
vgames = vgames.sort_values(by='Year')
vgames

In [ ]:
first_nagenre_index = first_vgames[first_vgames['Genre'].isna()==1].index
first_napublisher_index = first_vgames[first_vgames['Publisher'].isna()==1].index

In [ ]:
first_nagenre_index

In [ ]:
vgames

In [ ]:
vgames[vgames['Name'] == vgames.loc[3]['Name']]

In [ ]:
for i in first_nagenre_index:
  name = vgames.loc[i]['Name']
  temp_df = vgames[vgames['Name'] == name]
  temp_df = temp_df.sort_values(by=['Genre'],na_position='last')
  if len(temp_df) >=2:
    vgames.at[i,'Genre'] = temp_df.iloc[0]['Genre']

In [ ]:
for i in first_napublisher_index:
  name = vgames.loc[i]['Name']
  temp_df = vgames[vgames['Name'] == name]
  temp_df = temp_df.sort_values(by=['Publisher'],na_position='last')
  if len(temp_df) >=2:
    vgames.at[i,'Publisher'] = temp_df.iloc[0]['Publisher']

In [ ]:
vgames.isna().sum()

### 장르 최빈값으로 결측값 채우기

In [ ]:
vgames[vgames['Genre'].isnull()]

In [ ]:
genre_naindex = vgames[vgames['Genre'].isnull()].index
genre_naindex

In [ ]:
# Genre NAN 1
print(vgames.loc[genre_naindex[0],'Name'])
print(vgames[vgames['Name'].str.contains('Racing')]['Genre'].mode()[0])
vgames.at[genre_naindex[0],'Genre'] = vgames[vgames['Name'].str.contains('Racing')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[0],'Genre'])

In [ ]:
# Genre NAN 2
print(vgames.loc[genre_naindex[1],'Name'])
print(vgames[vgames['Name'].str.contains('Chess')]['Genre'].mode()[0])
vgames.at[genre_naindex[1],'Genre'] = vgames[vgames['Name'].str.contains('Chess')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[1],'Genre'])

In [ ]:
# Genre NAN 3
print(vgames.loc[genre_naindex[2],'Name'])
print(vgames[vgames['Name'].str.contains('F-Zero')]['Genre'].mode()[0])
vgames.at[genre_naindex[2],'Genre'] = vgames[vgames['Name'].str.contains('F-Zero')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[2],'Genre'])

In [ ]:
# Genre NAN 4
print(vgames.loc[genre_naindex[3],'Name'])
print(vgames[vgames['Name'].str.contains('Chrono')]['Genre'].mode()[0])
vgames.at[genre_naindex[3],'Genre'] = vgames[vgames['Name'].str.contains('Chrono')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[3],'Genre'])

In [ ]:
# Genre NAN 5
print(vgames.loc[genre_naindex[4],'Name'])
print(vgames[vgames['Name'].str.contains('Urban Chaos')]['Genre'].mode()[0])
vgames.at[genre_naindex[4],'Genre'] = vgames[vgames['Name'].str.contains('Urban Chaos')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[4],'Genre'])

In [ ]:
# Genre NAN 6
print(vgames.loc[genre_naindex[5],'Name'])
print(vgames[vgames['Name'].str.contains('Jurassic Park III')]['Genre'].mode()[0])
vgames.at[genre_naindex[5],'Genre'] = vgames[vgames['Name'].str.contains('Jurassic Park III')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[5],'Genre'])

In [ ]:
# Genre NAN 7
print(vgames.loc[genre_naindex[6],'Name'])
print(vgames[vgames['Name'].str.contains('Onimusha')]['Genre'].mode()[0])
vgames.at[genre_naindex[6],'Genre'] = vgames[vgames['Name'].str.contains('Onimusha')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[6],'Genre'])

In [ ]:
# Genre NAN 8
print(vgames.loc[genre_naindex[7],'Name'])
print(vgames[vgames['Name'].str.contains('Commando')]['Genre'].mode()[0])
vgames.at[genre_naindex[7],'Genre'] = vgames[vgames['Name'].str.contains('Commando')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[7],'Genre'])

In [ ]:
# Genre NAN 9
print(vgames.loc[genre_naindex[8],'Name'])
print(vgames[vgames['Name'].str.contains('Medabots')]['Genre'].mode()[0])
vgames.at[genre_naindex[8],'Genre'] = vgames[vgames['Name'].str.contains('Medabots')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[8],'Genre'])

In [ ]:
# Genre NAN 10
print(vgames.loc[genre_naindex[9],'Name'])
print(vgames[vgames['Name'].str.contains('WarioWare')]['Genre'].mode()[0])
vgames.at[genre_naindex[9],'Genre'] = vgames[vgames['Name'].str.contains('WarioWare')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[9],'Genre'])

In [ ]:
# Genre NAN 11
print(vgames.loc[genre_naindex[10],'Name'])
print(vgames[vgames['Name'].str.contains('Ape Escape')]['Genre'].mode()[0])
vgames.at[genre_naindex[10],'Genre'] = vgames[vgames['Name'].str.contains('Ape Escape')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[10],'Genre'])

In [ ]:
# Genre NAN 12
print(vgames.loc[genre_naindex[11],'Name'])
print(vgames[vgames['Name'].str.contains('Brain Boost')]['Genre'].mode()[0])
vgames.at[genre_naindex[11],'Genre'] = vgames[vgames['Name'].str.contains('Brain Boost')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[11],'Genre'])

In [ ]:
# Genre NAN 13
print(vgames.loc[genre_naindex[12],'Name'])
print(vgames[vgames['Name'].str.contains('Story')]['Genre'].mode()[0])
vgames.at[genre_naindex[12],'Genre'] = vgames[vgames['Name'].str.contains('Story')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[12],'Genre'])

In [ ]:
# Genre NAN 14
print(vgames.loc[genre_naindex[13],'Name'])
print(vgames[vgames['Name'].str.contains('Final Fantasy')]['Genre'].mode()[0])
vgames.at[genre_naindex[13],'Genre'] = vgames[vgames['Name'].str.contains('Final Fantasy')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[13],'Genre'])

In [ ]:
# Genre NAN 15
print(vgames.loc[genre_naindex[14],'Name'])
print(vgames[vgames['Name'].str.contains('PQ')]['Genre'].mode()[0])
vgames.at[genre_naindex[14],'Genre'] = vgames[vgames['Name'].str.contains('PQ')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[14],'Genre'])

In [ ]:
# Genre NAN 16
print(vgames.loc[genre_naindex[15],'Name'])
print(vgames[vgames['Name'].str.contains('Kart')]['Genre'].mode()[0])
vgames.at[genre_naindex[15],'Genre'] = vgames[vgames['Name'].str.contains('Kart')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[15],'Genre'])

In [ ]:
# Genre NAN 17
print(vgames.loc[genre_naindex[16],'Name'])
print(vgames[vgames['Name'].str.contains('The Godfather')]['Genre'].mode()[0])
vgames.at[genre_naindex[16],'Genre'] = vgames[vgames['Name'].str.contains('The Godfather')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[16],'Genre'])

In [ ]:
# Genre NAN 18
print(vgames.loc[genre_naindex[17],'Name'])
print(vgames[vgames['Name'].str.contains('UFO')]['Genre'].mode()[0])
vgames.at[genre_naindex[17],'Genre'] = vgames[vgames['Name'].str.contains('UFO')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[17],'Genre'])

In [ ]:
# Genre NAN 19
print(vgames.loc[genre_naindex[18],'Name'])
print(vgames[vgames['Name'].str.contains('Myth Makers')]['Genre'].mode()[0])
vgames.at[genre_naindex[18],'Genre'] = vgames[vgames['Name'].str.contains('Myth Makers')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[18],'Genre'])

In [ ]:
# Genre NAN 20
print(vgames.loc[genre_naindex[19],'Name'])
print(vgames[vgames['Name'].str.contains('SingStar')]['Genre'].mode()[0])
vgames.at[genre_naindex[19],'Genre'] = vgames[vgames['Name'].str.contains('SingStar')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[19],'Genre'])

In [ ]:
# Genre NAN 21
print(vgames.loc[genre_naindex[20],'Name'])
print(vgames[vgames['Name'].str.contains('Brain Quest')]['Genre'].mode()[0])
vgames.at[genre_naindex[20],'Genre'] = vgames[vgames['Name'].str.contains('Brain Quest')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[20],'Genre'])

In [ ]:
# Genre NAN 22
print(vgames.loc[genre_naindex[21],'Name'])
print(vgames[vgames['Name'].str.contains('Puppy')]['Genre'].mode()[0])
vgames.at[genre_naindex[21],'Genre'] = vgames[vgames['Name'].str.contains('Puppy')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[21],'Genre'])

In [ ]:
# Genre NAN 23
print(vgames.loc[genre_naindex[22],'Name'])
print(vgames[vgames['Name'].str.contains('Yakuza')]['Genre'].mode()[0])
vgames.at[genre_naindex[22],'Genre'] = vgames[vgames['Name'].str.contains('Yakuza')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[22],'Genre'])

In [ ]:
# Genre NAN 24
print(vgames.loc[genre_naindex[23],'Name'])
print(vgames[vgames['Name'].str.contains('Party')]['Genre'].mode()[0])
vgames.at[genre_naindex[23],'Genre'] = vgames[vgames['Name'].str.contains('Party')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[23],'Genre'])

In [ ]:
# Genre NAN 25
print(vgames.loc[genre_naindex[24],'Name'])
print(vgames[vgames['Name'].str.contains('Princess')]['Genre'].mode()[0])
vgames.at[genre_naindex[24],'Genre'] = vgames[vgames['Name'].str.contains('Princess')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[24],'Genre'])

In [ ]:
# Genre NAN 26
print(vgames.loc[genre_naindex[25],'Name'])
print(vgames[vgames['Name'].str.contains('Treasure')]['Genre'].mode()[0])
vgames.at[genre_naindex[25],'Genre'] = vgames[vgames['Name'].str.contains('Treasure')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[25],'Genre'])

In [ ]:
# Genre NAN 27
print(vgames.loc[genre_naindex[26],'Name'])
print(vgames[vgames['Name'].str.contains('Portable')]['Genre'].mode()[0])
vgames.at[genre_naindex[26],'Genre'] = vgames[vgames['Name'].str.contains('Portable')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[26],'Genre'])

In [ ]:
# Genre NAN 28
print(vgames.loc[genre_naindex[27],'Name'])
print(vgames[vgames['Name'].str.contains('Pokemon')]['Genre'].mode()[0])
vgames.at[genre_naindex[27],'Genre'] = vgames[vgames['Name'].str.contains('Pokemon')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[27],'Genre'])

In [ ]:
# Genre NAN 29
print(vgames.loc[genre_naindex[28],'Name'])
print(vgames[vgames['Name'].str.contains('Blood')]['Genre'].mode()[0])
vgames.at[genre_naindex[28],'Genre'] = vgames[vgames['Name'].str.contains('Blood')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[28],'Genre'])

In [ ]:
# Genre NAN 30
print(vgames.loc[genre_naindex[29],'Name'])
print(vgames[vgames['Name'].str.contains('Ukiyo')]['Genre'].mode()[0])
vgames.at[genre_naindex[29],'Genre'] = vgames[vgames['Name'].str.contains('Ukiyo')]['Genre'].mode()[0]
print(vgames.at[genre_naindex[29],'Genre'])

In [ ]:
vgames.isna().sum()

### Year 결측값 채우기 - 플랫폼마다 평균 연도는 다를 것이다.

In [ ]:
platform_names = vgames['Platform'].unique()
platform_names

In [ ]:
num = 0
group = dict()
for i in range(len(platform_names)):
  group[num] = 0
  group[num] = np.array(vgames[(vgames['Platform']== platform_names[i])&(vgames['Year'].notnull())]['Year'])
  num = num+1

In [ ]:
F_statistic, pVal = stats.f_oneway(group[0],group[1],group[2],group[3],
                                   group[4],group[5],group[6],group[7],
                                   group[8],group[9],group[10],group[11],
                                   group[12],group[13],group[14],group[15],
                                   group[16],group[17],group[18],group[19],
                                   group[20],group[21],group[22],group[23],
                                   group[24],group[25],group[26],group[27],
                                   group[28],group[29])

print('플랫폼의 연도 일원분산분석 결과 : F={0:.1f}, p={1:.5f}'.format(F_statistic, pVal))
if pVal < 0.05:
    print('P-value 값이 충분히 작음으로 인해 그룹의 평균값이 통계적으로 유의미하게 차이납니다.')

In [ ]:
nayear = vgames[vgames['Year'].isna()]
nayear_index = nayear.index
nayear_index

In [ ]:
platform_yearmean = pd.DataFrame(vgames.groupby('Platform')['Year'].mean())
platform_yearmean = platform_yearmean.apply(round)
platform_yearmean

In [ ]:
for i in nayear_index:
  platform_name = vgames.loc[i,'Platform']
  vgames.at[i,'Year'] = platform_yearmean.loc[platform_name]

In [ ]:
vgames.isna().sum()

# 2.지역에 따라서 선호하는 게임 장르가 다를까


In [ ]:
# 귀무가설: 지역에 따른 선호하는 게임 장르가 독립적이다.
# 대립가설: 지역에 따른 선호하는 게임 장르가 독립적이지 않다.

In [ ]:
a = pd.DataFrame(vgames['NA_Sales'].groupby(vgames["Genre"]).sum())
b = pd.DataFrame(vgames['EU_Sales'].groupby(vgames["Genre"]).sum())
c = pd.DataFrame(vgames['JP_Sales'].groupby(vgames["Genre"]).sum())
d = pd.DataFrame(vgames['Other_Sales'].groupby(vgames["Genre"]).sum())

ab = pd.merge(a,b,left_index=True, right_index=True,how='outer')
cd = pd.merge(c,d,left_index=True, right_index=True,how='outer')
abcd = pd.merge(ab,cd,left_index=True, right_index=True,how='outer')
abcd

## 분석

In [ ]:
# χ2 - test 가설 검정 
chipv = chi2_contingency(abcd, correction = True)
print("카이제곱 검정 결과 p-value가",round(chipv[1],4),"가 나왔으므로, \n지역에 따라 선호하는 게임 장르가 독립적인지에 대한 귀무가설을 유의수준 5%에서 기각한다.")
print("따라서 지역에 따른 선호 게임 장르가 독립적이지 않다는 대립가설을 채택한다.")

In [ ]:
print("미국과 유럽과 기타 지역 Action Sports Shooter")
print("일본 Role-Playing Action Sports ")

# 3.연도별 게임의 트렌드가 있을까


In [ ]:
vgames

In [ ]:
vgames_anal2 = vgames.sort_values(by='Year')
vgames_anal2

In [ ]:
year_genre = pd.DataFrame(vgames_anal2.groupby('Year')['Genre'].value_counts())
year_genre

In [ ]:
genre_year = pd.DataFrame(vgames_anal2.groupby('Genre')['Year'].value_counts())
genre_year

In [ ]:
anal2_unstack = genre_year.unstack(level=0)
anal2_unstack = anal2_unstack.fillna(0)
anal2_unstack.head()

In [ ]:
year_genre_unstack = year_genre.unstack()
year_genre_unstack = year_genre_unstack.fillna(0)
year_genre_unstack.head()

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(anal2_unstack.index, anal2_unstack.Year)
plt.title('Line Graph Year Genre', fontsize=20) 

plt.ylabel('Cummulative Num', fontsize=14)

plt.xlabel('Year', fontsize=14)

plt.legend(fontsize=12, loc='best')

plt.show()

In [ ]:
yeargenre_sum = pd.DataFrame(vgames.groupby(['Year','Genre']).sum().sum(axis=1)).unstack()
yeargenre_sum = yeargenre_sum.fillna(0)
yeargenre_sum.head()

In [ ]:
# χ2 - test 가설 검정 
chipv3 = chi2_contingency(yeargenre_sum, correction = True)
print("카이제곱 검정 결과 p-value가",round(chipv3[1],4),"가 나왔으므로, \n연도에 따른 게임 장르가 독립적인지에 대한 귀무가설을 유의수준 5%에서 기각한다.")
print("따라서 연도에 따른 게임 장르가 독립적이지 않다는 대립가설을 채택한다.")

In [ ]:
genre_names = vgames['Genre'].unique()
genre_names

In [ ]:
plt.figure(figsize=(20,5))
for i in genre_names:
  plt.plot(yeargenre_sum.index, yeargenre_sum[(0,i)])
  plt.title('Line Graph Year Genre', fontsize=20) 
  plt.ylabel('Total Sales', fontsize=14)
  plt.xlabel('Year', fontsize=14)
plt.show()

In [ ]:
grouped = pd.DataFrame(vgames.groupby(['Year','Genre']).sum().sum(axis=1))
ggg = grouped.unstack().fillna(0)
ggg.head()

In [ ]:
for i in ggg.index:
  temp_df = ggg.loc[i].sort_values(ascending=False)
  max_index = temp_df.index[0][1]
  print(i,max_index)

In [ ]:
max_index = []
for i in ggg.index:
  temp_df = ggg.loc[i].sort_values(ascending=False)
  max_index.append(temp_df.index[0][1])

In [ ]:
max_dict = dict()

In [ ]:
for i in max_index:
  if i not in max_dict:
    max_dict[i] = 1
  else:
    max_dict[i] = max_dict[i]+1

In [ ]:
max_dict

## 분석
-  카이제곱 검정을 통해 연도에 따른 게임 장르가 독립적이지 않다는 가설을 채택한다.
= 연도별 선호하는 게임 장르가 다를 것이다.
- 하지만 부동의 1위 Action 장르

# 4.출고량이 높은 게임에 대한 Analysis 및 Visualization

In [ ]:
vgames = vgames.sort_values(by='Year')
vgames = vgames.reset_index()
vgames = vgames.iloc[:,1:]
vgames

In [ ]:
vgames['Total'] = vgames['NA_Sales'] + vgames['JP_Sales'] + vgames['EU_Sales'] + vgames['Other_Sales']
vgames.head()

In [ ]:
anal3 = vgames.sort_values(by='Total',ascending=False)
anal3 = anal3.reset_index()
anal3 = anal3.iloc[:,1:]
anal3

## -1980~2016 Top10 & Top100

In [ ]:
anal3_top10 = anal3.iloc[:10,]
print('*'*8,'Top 10 Name','*'*8 )
print(anal3_top10['Name'])
print()
print('*'*8,'Top 10 Genre','*'*8 )
print(anal3_top10['Genre'])
print()
print('*'*8,'Top 10 Genre Count','*'*8)
print(anal3_top10['Genre'].value_counts())

print()
print('*'*8,'Top 10 Platform Count','*'*8)
print(anal3_top10['Platform'].value_counts())
print()
print('*'*8,'Top 10 Publisher Count','*'*8)
print(anal3_top10['Publisher'].value_counts())
print()
print('*'*8,'Top 10 Total Sales','*'*8)
anal3_top10count = pd.DataFrame(anal3_top10.groupby('Genre')['Total'].sum()).sort_values(by='Total',ascending=False)
print(anal3_top10count)
plt.figure(figsize=(10,4))
sns.barplot(x=anal3_top10count.index,y='Total',data=anal3_top10count)

### Top 10 해석


1980년부터 2016까지의 판매량 Top 10 
1. Wii Sports
2. Super Mario Bros.
3. Mario Kart Wii

모두 Nintendo에서 나왔다.

장르는 스포츠, 플랫폼, 레이싱 순으로 출고량이 많다.


In [ ]:
anal3_top100 = anal3.iloc[:100,]
print('*'*8,'Top 100 Genre','*'*8 )
print(anal3_top100['Genre'])
print()
print('*'*8,'Top 100 Genre Count','*'*8)
print(anal3_top100['Genre'].value_counts())
anal3_top100_counts = pd.DataFrame(anal3_top100['Genre'].value_counts())
plt.figure(figsize=(10,4))
sns.barplot(x=anal3_top100_counts.index,y='Genre',data=anal3_top100_counts)
plt.show()
print()
print('*'*8,'Top 100 Total Sales','*'*8)
anal3_top100count = pd.DataFrame(anal3_top100.groupby('Genre')['Total'].sum()).sort_values(by='Total',ascending=False)
print(anal3_top100count)
plt.figure(figsize=(10,4))
sns.barplot(x=anal3_top100count.index,y='Total',data=anal3_top100count)
plt.show()
print('*'*8,'Top 100 Platform','*'*8)
anal3_top100countplatform = pd.DataFrame(anal3_top100.groupby('Platform')['Total'].sum()).sort_values(by='Total',ascending=False)
print(anal3_top100countplatform)
plt.figure(figsize=(10,4))
sns.barplot(x=anal3_top100countplatform.index,y='Total',data=anal3_top100countplatform)
plt.show()
print()
print('*'*8,'Top 100 Publisher','*'*8)
anla3_top100publisher = pd.DataFrame(anal3_top100.groupby('Publisher')['Total'].sum()).sort_values(by='Total',ascending=False)
print(anla3_top100publisher)
plt.figure(figsize=(30,4))
sns.barplot(x=anla3_top100publisher.index,y='Total',data=anla3_top100publisher)
plt.show()

### Top 100 해석

1980년부터 2016까지의 판매량 Top 100

[Genre]
1. Shooter
2. Platform
3. Role-Playing

위 3가지가 가장 많이 출시된 장르이며, 총 출고량도 가장 높다.

[Platform]
1. Wii
2. DS
3. X360

[Publisher]
1. Nintendo
2. Activision
3. Take-Two Interactive

Activision은 Call of Duty, Take-Two Interactive은 Grand Theft Auto
시리즈물의 판매량이 높아서 총 판매량 Publisher에 있음.

In [ ]:
totalsales = anal3['Total'].sum()
totalsales

In [ ]:
anal3_mariosales = vgames[vgames['Name'].str.contains('Mario')]['Total'].sum()
round((anal3_mariosales/totalsales)*100,2)

In [ ]:
anal3_taketwosales = anal3[anal3['Publisher']=='Take-Two Interactive']['Total'].sum()
round((anal3_taketwosales/totalsales)*100,2)

In [ ]:
anal3_callofdutysales = vgames[vgames['Name'].str.contains('Call of Duty')]['Total'].sum()
round((anal3_callofdutysales/totalsales)*100,2)

In [ ]:
anal3_pokemonsales = vgames[vgames['Name'].str.contains('Pokemon')]['Total'].sum()
round((anal3_pokemonsales/totalsales)*100,2)

In [ ]:
anal3_grandtheftsales = vgames[vgames['Name'].str.contains('Grand Theft Auto')]['Total'].sum()
round((anal3_grandtheftsales/totalsales)*100,2)

## -최근 5년 Top10 & Top100

In [ ]:
recent5year = vgames[vgames['Year']>=2011]
recent5year

In [ ]:
recent5year = recent5year.sort_values(by='Total',ascending=False)
recent5year = recent5year.reset_index()
recent5year = recent5year.iloc[:,1:]
recent5year

In [ ]:
print('*'*8,'Recent 5 years Top 10 Name','*'*8 )
recent5year_top10 = recent5year.iloc[:10,]
print()
print(recent5year_top10['Name'])
print('*'*8,'Recent 5 years Top 10 Genre','*'*8 )
print(recent5year_top10['Genre'])
print()
print('*'*8,'Recent 5 years Top 10 Genre Count','*'*8)
print(recent5year_top10['Genre'].value_counts())

print()
print('*'*8,'Recent 5 years Top 10 Platform Count','*'*8)
print(recent5year_top10['Platform'].value_counts())
print()
print('*'*8,'Recent 5 years Top 10 Publish Count','*'*8)
print(recent5year_top10['Publisher'].value_counts())
print()
print('*'*8,'Recent 5 years Top 10 Total Sales','*'*8)
recent5year_top10count = pd.DataFrame(recent5year_top10.groupby('Genre')['Total'].sum()).sort_values(by='Total',ascending=False)
print(recent5year_top100count)
plt.figure(figsize=(10,4))
sns.barplot(x=recent5year_top100count.index,y='Total',data=recent5year_top100count)

### Top10 해석

[Genre & Total Sales]
1. Shooter
2. Action
3. Role-Playing


[Publisher]
1. Activision
2. Take-Two Interactive
3. Nintendo


In [ ]:
recent5yeartotalsales = recent5year['Total'].sum()
recent5yeartotalsales

In [ ]:
recent5year_taketwosales = recent5year[recent5year['Publisher']=='Take-Two Interactive']['Total'].sum()
round((recent5year_taketwosales/recent5yeartotalsales)*100,2)

In [ ]:
recent5year_callofdutysales = recent5year[recent5year['Name'].str.contains('Call of Duty')]['Total'].sum()
round((recent5year_callofdutysales/recent5yeartotalsales)*100,2)

In [ ]:
recent5year_pokemonsales = recent5year[recent5year['Name'].str.contains('Pokemon')]['Total'].sum()
round((recent5year_pokemonsales/recent5yeartotalsales)*100,2)

In [ ]:
recent5year_grandtheftsales = recent5year[recent5year['Name'].str.contains('Grand Theft Auto')]['Total'].sum()
round((recent5year_grandtheftsales/recent5yeartotalsales)*100,2)

In [ ]:
recent5year_top100 = recent5year.iloc[:100,]
print('*'*8,'Recent 5 years Top 100 Genre','*'*8 )
print(recent5year_top100['Genre'])
print()
print('*'*8,'Recent 5 years Top 100 Genre Count','*'*8)
print(recent5year_top100['Genre'].value_counts())
recent5year_top100_counts = pd.DataFrame(recent5year_top100['Genre'].value_counts())
plt.figure(figsize=(10,4))
sns.barplot(x=recent5year_top100_counts.index,y='Genre',data=recent5year_top100_counts)
plt.show()
print()
print('*'*8,'Recent 5 years Top 100 Total Sales','*'*8)
b
plt.figure(figsize=(10,4))
sns.barplot(x=recent5year_top100count.index,y='Total',data=recent5year_top100count)
plt.show()
print()
print('*'*8,'Recent 5 years Top 100 Platform','*'*8)
recent5year_top100platform = pd.DataFrame(recent5year_top100.groupby('Platform')['Total'].sum()).sort_values(by='Total',ascending=False)
print(recent5year_top100platform)
plt.figure(figsize=(10,4))
sns.barplot(x=recent5year_top100platform.index,y='Total',data=recent5year_top100platform)
plt.show()
print()
print('*'*8,'Recent 5 years Top 100 Publisher','*'*8)
recent5year_top100publisher = pd.DataFrame(recent5year_top100.groupby('Publisher')['Total'].sum()).sort_values(by='Total',ascending=False)
print(recent5year_top100publisher)
plt.figure(figsize=(30,4))
sns.barplot(x=recent5year_top100publisher.index,y='Total',data=recent5year_top100publisher)
plt.show()

### Top 100 해석

[Genre]

1. Shooter
2. Platform
3. Role-Playing
위 3가지가 가장 많이 출시된 장르이며, 총 출고량도 가장 높다.
(최근 5년과 총 인기있는 장르는 동일하다.)

[Platform]

1. PS3 
2. X360
3. PS4 

[Publisher]

1. Nintendo
2. Activision
3. Electronic Arts 


In [ ]:
recent5year[(recent5year['Platform']=='PS3')&(recent5year['Publisher']=='Activision')]

# 5.결론

In [ ]:
'''
<분석 총 정리>
- 지역에 따라 선호하는 게임 장르가 다르다. 또한 미국과 유럽과 기타 지역은 Action Sports Shooter
일본 Role-Playing Action Sports 순으로 출고량이 많았다.
- 연도별 선호하는 게임 장르가 다르지만 부동의 1위 Action 장르.
- 1980년부터의 총 출고량과 최근 5년간 출고량이 많은 플랫폼을 보면 새로운 게임기기(Platform)에 맞춰 그에 맞는 게임이 인기가 많아진다.
또한 시리즈물의 인기는 사그라지지 않는 것을 확인할 수 있다.
'''

In [ ]:
'''
Goal: 다음 분기에 어떤 게임을 설계해야 할까?
장르는 Action 혹은 Sports,Shooter Nintendo 회사 게임을 설계하면 많이 팔릴 것 같다.
특히 유명 시리즈물인 마리오, 포켓몬, 콜오브듀티로 설계하면 매출을 올릴 가능성이 높다.
'''